## Enumeration 

Symbolic enumerations are similar to dictionaries and named tuples bc they map names/keys to values. The main difference is that the `Enum` defiition is __immutable__ and __global__. It should be used whenever there is a closed set of possible values that _cant change dynamically during program runtime_.

Dictionaries and named tuples are data containers - we can create as many as we like. 

[Expert Py Programming 3e](https://learning.oreilly.com/library/view/expert-python-programming/9781789808896/ee482d50-d26b-46ea-a318-5e974fb5449a.xhtml)

#### Symbolic enumeration with the enum module

In [1]:
from enum import Enum, auto

class Weekday(Enum):
    MONDAY = 0
    TUESDAY = 1
    WEDNESDAY = 2
    THURSDAY = 3
    FRIDAY = 4
    SATURDAY = auto()
    SUNDAY = auto()

In [2]:
f' - name is {Weekday.MONDAY.name} \
- value is {Weekday.MONDAY.value}'

' - name is MONDAY - value is 0'

use the `enum.auto()` function that cna replace values with automatically generated values. such as below

In [3]:
Weekday.SUNDAY

<Weekday.SUNDAY: 6>

In [6]:
from enum import Enum, auto


class OrderStatus(Enum):
    PENDING = auto()
    PROCESSING = auto()
    PROCESSED = auto()


class Order:
    def __init__(self):
        self.status = OrderStatus.PENDING

    def process(self):
        if self.status == OrderStatus.PROCESSED:
            raise RuntimeError(
                "Can't process order that has "
                "been already processed"
            )

        self.status = OrderStatus.PROCESSING
        ...
        self.status = OrderStatus.PROCESSED


In [12]:
k = Order()
k.__dict__

{'status': <OrderStatus.PENDING: 1>}

In [13]:
k.process()
k.__dict__

{'status': <OrderStatus.PROCESSED: 3>}

In [14]:
from enum import Flag, auto


class Side(Flag):
    GUACAMOLE = auto()
    TORTILLA = auto()
    FRIES = auto()
    BEER = auto()
    POTATO_SALAD = auto()

In [15]:
m = Side.GUACAMOLE | Side.BEER | Side.TORTILLA
m.__dict__

{'_name_': None, '_value_': 11}

In [19]:
Side.GUACAMOLE.value + Side.BEER.value + Side.TORTILLA.value

11

In [20]:
b = Side.BEER | Side.POTATO_SALAD
b.__dict__

{'_name_': None, '_value_': 24}

In [21]:
common_sides = m & b
common_sides.__dict__

{'_value_': 8, '_name_': 'BEER', '__objclass__': <enum 'Side'>}

## Iterators

and __Iterator__ is nothing more than a container object that impleents the iterator protocol. The protocols consist of 2 methods:
- `__next__` - return the next item of the containers
- `__iter__` - returns the iterator itself

__Iterators__ can be created from a sequence using the iter built-in function.

In [42]:
i = iter('abc')
print(next(i))
print(next(i))
print(next(i))
print(next(i))

a
b
c


StopIteration: 

When the sequence is exhausted, a StopIteration exception is raised.It makes iterators compatible with loops, since they catch this exception as a signal to end the iteration.

In [43]:
class CountDown:
    def __init__(self, step):
        self.step = step

    def __next__(self):
        """Return the next element."""
        if self.step <= 0:
            raise StopIteration
        self.step -= 1
        return self.step

    def __iter__(self):
        """Return the iterator itself."""
        return self

In [47]:
count_down = CountDown(4)
for element in count_down:
    print(element)


3
2
1
0


#### Generators and yield statements

__Generators__ provide an efficient way to write simple code for functions that return a sequence of elements. Based on the `yield` statement, they allow functions to be paused and return an intermediate result. The function saevs its execution context and can be resumed later if necessary. 

__Details on Generators__
- Generators functions contains one or more `yield` statements
- When called, it returns an object (iterator) but does not start execution immediately
- Methods like `__iter__()` and `__next__()` are implemented automatically. So we can iterate thorugh the items using `next()`
- Once the function yields, the funciton is paused and the control is transfered to the _caller_
- Local valriables and their states are rememebered between successive calles
- Finally, when the funciton terminates, `StopIteration` is raised automatically on futher calls.

_https://www.programiz.com/python-programming/generator_


__Below is a imple generator example__

In [48]:
# A simple generator function
def my_gen():
    n = 1
    print('This is printed first')
    # Generator function contains yield statements
    yield n

    n += 1
    print('This is printed second')
    yield n

    n += 1
    print('This is printed at last')
    yield n

In [50]:
# It returns an object but does not start execution immediately.
a = my_gen()
a

<generator object my_gen at 0x7fc884574de0>

In [53]:
# We can iterate through the items using next().
next(a)

# Once the function yields, the function is paused and the control is transferred to the caller.

This is printed first


1

In [54]:
# Local variables and theirs states are remembered between successive calls.
next(a)
next(a)

This is printed second
This is printed at last


3

In [55]:
next(a)

StopIteration: 

Unlike _normal funcitons_ where the __local variables are not destroyed__ when the functinon yields. Here above using the __Generators__ the value of the variable `n` is remember between each call. To restart the process we just have to recreate the generator `a = my_gen()`

__Generators are used with Loops__ includinga terminating condition

In [57]:
# Using for loop
for item in my_gen():
    print(item)

This is printed first
1
This is printed second
2
This is printed at last
3


In [78]:
# Initialize the list
my_list = [1, 3, 6, 10]

# square each term using list comprehension
list_ = [x**2 for x in my_list]

# same thing can be done using generator expression
generator = (x**2 for x in my_list)

print(list_)
print(generator)

for item in generator:
    print(item)

[1, 9, 36, 100]
<generator object <genexpr> at 0x7fc88c44c048>
1
9
36
100


We can see above that the generator expression did not produce the required result immediately. Instead, it returned a generator object, which produces items only on demand.

In [75]:
next(a)

TypeError: 'int' object is not an iterator

###### Generators Rerpresent Infinite Streams & Memory Efficient

__Generators__ are the best at representing an infinite stream of data. Typically infinite streams cannot be stored in memory, which is a great application for generators because __generators produce only one item at a time__ therefore can represent an infiniate stream of data. 

__Memory Efficient__ - a normal function stors the entire sequence in memory before returning the result. This is not practical if the sequence is large. __Generators__ provides a memory efficient representaiton as __Generators only produce one item at a time__ 

In [79]:
def all_even():
    n = 0
    while True:
        yield n
        n += 2

In [113]:
e = all_even()
[next(e) for i in range(10)]

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]

In [125]:
def PowTwoGen(max=0):
    n = 0
    while n < max:
        yield 2 ** n
        n += 1
        
k = PowTwoGen(10)
[print(next(k)) for i in range(9)]
print("final item is: ", next(k))

1
2
4
8
16
32
64
128
256
final item is:  512


---

### Decorators


__Decorators__ take a funciton, adds additional functionality and returns it. This is masically adding a _meta-function_ that is used to modify another part of the function at complie time. 

Below is a simple example of a basic function that we will later add the Decorators. 


_https://www.programiz.com/python-programming/decorator_

In [128]:
def first(msg):
    print(msg)    

first("Hello")

second = first
second("Hello")

Hello
Hello


Above we ran `first()` and `second()` that returns the same output, and both refer to the same `first()` function object. 

Now we introduce a concept similar to `decorators` which is __Higher order functions__ this is the technical name for the example below were wewe will use a funciton to be passed as a argument to another function.

In [132]:
def inc(x):
    return x + 1

def dec(x):
    return x - 1

def operate(func, x):
    result = func(x)
    return result

print(operate(inc, 3))
print(operate(dec, 3))

4
2


example of a __funciton return another function__ 

In [133]:
def is_called():
    def is_returned():
        print("Hello")
    return is_returned

new = is_called()

#Outputs "Hello"
new()

Hello


In [138]:
is_called()()

Hello


Back to __Decorators__ - recall __decorators take a funciton, adds funcitonality and returns it__

In [174]:
def make_pretty(func):
    def inner():
        print("I got decorated")
        func()
    return inner

def ordinary():
    print("I am ordinary")

In [140]:
ordinary()

I am ordinary


Here we add the dectorator `make_pretty` to _"decorate"_ the `ordinary()`. The function `ordinary()` got decorated and the returned function was given the name `pretty`.

In [146]:
pretty = make_pretty(ordinary)
print(pretty)

<function make_pretty.<locals>.inner at 0x7fc8841881e0>


In [147]:
pretty()

I got decorated
I am ordinary


_like a Christmas Gift_ the __`decorator`__ acts as a _"gift wrapper"_ to decorate the present `original()`. The nature of the object that got decorated (actual gift inside) does not alter. But now, it looks pretty (since it got decorated).

Generally, we decorate a function and reassign it as,

In [176]:
pretty_ordinary = make_pretty(ordinary)
pretty_ordinary()

I got decorated
I am ordinary


__Formal use of @decorators__

The `@` is used with the name of the decorator function placed above the funciton to be decorated, like so:


which is equivilent to what we did above:

```python
def ordinary():
    print("I am ordinary")
ordinary = make_pretty(ordinary)

```

In [179]:
@make_pretty
def ordinary():
    print("I am ordinary")
    
ordinary()

I got decorated
I am ordinary


In [194]:
def smart_divide(func):
   def inner(*args, **kwargs):
      print("I am going to divide",args[0],"and",args[1])
      if args[1] == 0:
         print("Whoops! cannot divide")
         return

      return func(*args, **kwargs)
   return inner

@smart_divide
def divide(a,b):
    return a/b

In [195]:
divide(2,5)

I am going to divide 2 and 5


0.4

In [196]:
divide(2,0)

I am going to divide 2 and 0
Whoops! cannot divide
